In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
from matplotlib.patches import Patch
from pathlib import Path
from multiprocessing.dummy import Pool as ThreadPool
from collections import defaultdict
from natsort import natsorted
import tsfresh as tf
import sklearn

In [2]:
pd.set_option('max_columns', None)

In [3]:
import os
import glob

dataset_path = 'data_feats_90_c/'
csv_files = glob.glob(dataset_path+'*.csv')

all_df = []

for filename in csv_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    all_df.append(df)

df = pd.concat(all_df, axis=0, ignore_index=True)

In [4]:
df = df[df.class_code != 7]
df['class_code'].replace({8: 7}, inplace=True)


In [5]:
df.class_code.unique()

array([5, 7, 3, 1, 4, 6, 0, 2])

In [6]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    nans = df.isna().sum(axis=0)
    print(nans[nans!=0])

P-PDG__fft_aggregated__aggtype_"centroid"                 8571
P-PDG__fft_aggregated__aggtype_"variance"                 8571
P-PDG__fft_aggregated__aggtype_"skew"                    37244
P-PDG__fft_aggregated__aggtype_"kurtosis"                37244
P-PDG__variation_coefficient                              8571
P-PDG__friedrich_coefficients__coeff_1__m_3__r_30        20675
P-PDG__friedrich_coefficients__coeff_3__m_3__r_30        20675
P-TPT__fft_aggregated__aggtype_"centroid"                   64
P-TPT__fft_aggregated__aggtype_"variance"                   64
P-TPT__fft_aggregated__aggtype_"skew"                    30450
P-TPT__fft_aggregated__aggtype_"kurtosis"                30450
P-TPT__variation_coefficient                                64
P-TPT__friedrich_coefficients__coeff_1__m_3__r_30        12623
P-TPT__friedrich_coefficients__coeff_3__m_3__r_30        12623
T-TPT__fft_aggregated__aggtype_"centroid"                 6514
T-TPT__fft_aggregated__aggtype_"variance"              

In [7]:
df.drop(['Unnamed: 0'], axis='columns', inplace=True) #leftover index column after feat. extrs, unnecessary
df.drop(['QGL__variation_coefficient'], axis='columns', inplace=True) #too many nan values
df.drop(list(df.filter(regex = 'friedrich')), axis = 1, inplace = True)
df.drop(list(df.filter(regex = 'fft_aggregated')), axis = 1, inplace = True)

In [8]:
print(df.shape)
df.head(1)

(51764, 108)


,P-PDG__mean,P-PDG__variance,P-PDG__skewness,P-PDG__kurtosis,P-PDG__maximum,P-PDG__minimum,P-PDG__median,P-PDG__quantile__q_0.1,P-PDG__quantile__q_0.2,P-PDG__quantile__q_0.3,P-PDG__quantile__q_0.4,P-PDG__quantile__q_0.6,P-PDG__quantile__q_0.7,P-PDG__quantile__q_0.8,P-PDG__quantile__q_0.9,P-PDG__variation_coefficient,P-PDG__mean_change,P-PDG__mean_second_derivative_central,P-TPT__mean,P-TPT__variance,P-TPT__skewness,P-TPT__kurtosis,P-TPT__maximum,P-TPT__minimum,P-TPT__median,P-TPT__quantile__q_0.1,P-TPT__quantile__q_0.2,P-TPT__quantile__q_0.3,P-TPT__quantile__q_0.4,P-TPT__quantile__q_0.6,P-TPT__quantile__q_0.7,P-TPT__quantile__q_0.8,P-TPT__quantile__q_0.9,P-TPT__variation_coefficient,P-TPT__mean_change,P-TPT__mean_second_derivative_central,T-TPT__mean,T-TPT__variance,T-TPT__skewness,T-TPT__kurtosis,T-TPT__maximum,T-TPT__minimum,T-TPT__median,T-TPT__quantile__q_0.1,T-TPT__quantile__q_0.2,T-TPT__quantile__q_0.3,T-TPT__quantile__q_0.4,T-TPT__quantile__q_0.6,T-TPT__quantile__q_0.7,T-TPT__quantile__q_0.8,T-TPT__quantile__q_0.9,T-TPT__variation_coefficient,T-TPT__mean_change,T-TPT__mean_second_derivative_central,P-MON-CKP__mean,P-MON-CKP__variance,P-MON-CKP__skewness,P-MON-CKP__kurtosis,P-MON-CKP__maximum,P-MON-CKP__minimum,P-MON-CKP__median,P-MON-CKP__quantile__q_0.1,P-MON-CKP__quantile__q_0.2,P-MON-CKP__quantile__q_0.3,P-MON-CKP__quantile__q_0.4,P-MON-CKP__quantile__q_0.6,P-MON-CKP__quantile__q_0.7,P-MON-CKP__quantile__q_0.8,P-MON-CKP__quantile__q_0.9,P-MON-CKP__variation_coefficient,P-MON-CKP__mean_change,P-MON-CKP__mean_second_derivative_central,T-JUS-CKP__mean,T-JUS-CKP__variance,T-JUS-CKP__skewness,T-JUS-CKP__kurtosis,T-JUS-CKP__maximum,T-JUS-CKP__minimum,T-JUS-CKP__median,T-JUS-CKP__quantile__q_0.1,T-JUS-CKP__quantile__q_0.2,T-JUS-CKP__quantile__q_0.3,T-JUS-CKP__quantile__q_0.4,T-JUS-CKP__quantile__q_0.6,T-JUS-CKP__quantile__q_0.7,T-JUS-CKP__quantile__q_0.8,T-JUS-CKP__quantile__q_0.9,T-JUS-CKP__variation_coefficient,T-JUS-CKP__mean_change,T-JUS-CKP__mean_second_derivative_central,QGL__mean,QGL__variance,QGL__skewness,QGL__kurtosis,QGL__maximum,QGL__minimum,QGL__median,QGL__quantile__q_0.1,QGL__quantile__q_0.2,QGL__quantile__q_0.3,QGL__quantile__q_0.4,QGL__quantile__q_0.6,QGL__quantile__q_0.7,QGL__quantile__q_0.8,QGL__quantile__q_0.9,QGL__mean_change,QGL__mean_second_derivative_central,class_code
0,3.045252e+07,39.064321,-0.029904,-1.707532,30452532.0,30452512.0,30452523.0,30452514.0,30452515.0,30452516.0,30452518.0,30452526.0,30452527.3,30452528.0,30452529.0,2.052423e-07,-0.168539,0.0,2.145328e+07,12899.168025,0.19382,-0.722604,21453566.0,21453039.0,21453275.0,21453131.0,21453169.0,21453199.7,21453229.6,21453331.0,21453348.3,21453372.2,21453425.2,0.000005,3.022472,1.579545,125.35756,7.086469e-09,0.054108,-0.473179,125.35778,125.35736,125.35756,125.35745,125.35748,125.357517,125.357536,125.35758,125.35761,125.357632,125.357661,6.715285e-07,-7.865169e-07,6.250000e-07,1519597.66,3.915511,-0.008192,-1.022432,1519601.0,1519594.0,1519597.7,1519595.0,1519595.98,1519596.37,1519597.0,1519598.0,1519599.0,1519599.82,1519600.31,0.000001,-0.078652,0.0,92.644159,0.000002,0.059343,-1.201477,92.646534,92.641924,92.64413,92.642336,92.64277,92.643215,92.64367,92.644594,92.645066,92.645547,92.646035,0.000015,-0.000052,6.250000e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5


In [9]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    nans = df.isna().sum(axis=0)
    print(nans[nans!=0])

P-PDG__variation_coefficient        8571
P-TPT__variation_coefficient          64
T-TPT__variation_coefficient        6514
P-MON-CKP__variation_coefficient    1200
T-JUS-CKP__variation_coefficient    1819
dtype: int64


In [10]:
imputed_df_list = []
for i in df['class_code'].unique():
    query = 'class_code == ' + str(i)
    print(query)
    imputed = tf.utilities.dataframe_functions.impute(df.query(query))
    print(imputed.shape)
    imputed_df_list.append(imputed)
imputed_df = pd.concat(imputed_df_list)
imputed_df

class_code == 5


/home/marko/.local/lib/python3.8/site-packages/pandas/core/generic.py:9032: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(cond, other, inplace, axis, level, errors=errors)


(14443, 108)
class_code == 7
(2529, 108)
class_code == 3
(5284, 108)
class_code == 1
(9299, 108)
class_code == 4
(2468, 108)
class_code == 6
(6508, 108)
class_code == 0
(10556, 108)
class_code == 2
(677, 108)


,P-PDG__mean,P-PDG__variance,P-PDG__skewness,P-PDG__kurtosis,P-PDG__maximum,P-PDG__minimum,P-PDG__median,P-PDG__quantile__q_0.1,P-PDG__quantile__q_0.2,P-PDG__quantile__q_0.3,P-PDG__quantile__q_0.4,P-PDG__quantile__q_0.6,P-PDG__quantile__q_0.7,P-PDG__quantile__q_0.8,P-PDG__quantile__q_0.9,P-PDG__variation_coefficient,P-PDG__mean_change,P-PDG__mean_second_derivative_central,P-TPT__mean,P-TPT__variance,P-TPT__skewness,P-TPT__kurtosis,P-TPT__maximum,P-TPT__minimum,P-TPT__median,P-TPT__quantile__q_0.1,P-TPT__quantile__q_0.2,P-TPT__quantile__q_0.3,P-TPT__quantile__q_0.4,P-TPT__quantile__q_0.6,P-TPT__quantile__q_0.7,P-TPT__quantile__q_0.8,P-TPT__quantile__q_0.9,P-TPT__variation_coefficient,P-TPT__mean_change,P-TPT__mean_second_derivative_central,T-TPT__mean,T-TPT__variance,T-TPT__skewness,T-TPT__kurtosis,T-TPT__maximum,T-TPT__minimum,T-TPT__median,T-TPT__quantile__q_0.1,T-TPT__quantile__q_0.2,T-TPT__quantile__q_0.3,T-TPT__quantile__q_0.4,T-TPT__quantile__q_0.6,T-TPT__quantile__q_0.7,T-TPT__quantile__q_0.8,T-TPT__quantile__q_0.9,T-TPT__variation_coefficient,T-TPT__mean_change,T-TPT__mean_second_derivative_central,P-MON-CKP__mean,P-MON-CKP__variance,P-MON-CKP__skewness,P-MON-CKP__kurtosis,P-MON-CKP__maximum,P-MON-CKP__minimum,P-MON-CKP__median,P-MON-CKP__quantile__q_0.1,P-MON-CKP__quantile__q_0.2,P-MON-CKP__quantile__q_0.3,P-MON-CKP__quantile__q_0.4,P-MON-CKP__quantile__q_0.6,P-MON-CKP__quantile__q_0.7,P-MON-CKP__quantile__q_0.8,P-MON-CKP__quantile__q_0.9,P-MON-CKP__variation_coefficient,P-MON-CKP__mean_change,P-MON-CKP__mean_second_derivative_central,T-JUS-CKP__mean,T-JUS-CKP__variance,T-JUS-CKP__skewness,T-JUS-CKP__kurtosis,T-JUS-CKP__maximum,T-JUS-CKP__minimum,T-JUS-CKP__median,T-JUS-CKP__quantile__q_0.1,T-JUS-CKP__quantile__q_0.2,T-JUS-CKP__quantile__q_0.3,T-JUS-CKP__quantile__q_0.4,T-JUS-CKP__quantile__q_0.6,T-JUS-CKP__quantile__q_0.7,T-JUS-CKP__quantile__q_0.8,T-JUS-CKP__quantile__q_0.9,T-JUS-CKP__variation_coefficient,T-JUS-CKP__mean_change,T-JUS-CKP__mean_second_derivative_central,QGL__mean,QGL__variance,QGL__skewness,QGL__kurtosis,QGL__maximum,QGL__minimum,QGL__median,QGL__quantile__q_0.1,QGL__quantile__q_0.2,QGL__quantile__q_0.3,QGL__quantile__q_0.4,QGL__quantile__q_0.6,QGL__quantile__q_0.7,QGL__quantile__q_0.8,QGL__quantile__q_0.9,QGL__mean_change,QGL__mean_second_derivative_central,class_code
0,3.045252e+07,39.064321,-0.029904,-1.707532,30452532.0,30452512.0,30452523.0,30452514.0,30452515.0,30452516.0,30452518.0,30452526.0,30452527.3,30452528.0,30452529.0,2.052423e-07,-0.168539,0.000000,2.145328e+07,1.289917e+04,0.193820,-0.722604,21453566.0,21453039.0,21453275.0,21453131.0,21453169.0,21453199.7,21453229.6,21453331.0,21453348.3,21453372.2,21453425.2,0.000005,3.022472,1.579545,125.357560,7.086469e-09,0.054108,-0.473179,125.357780,125.357360,125.357560,125.357450,125.357480,125.357517,125.357536,125.357580,125.357610,125.357632,125.357661,6.715285e-07,-7.865169e-07,6.250000e-07,1.519598e+06,3.915511,-0.008192,-1.022432,1519601.0,1519594.0,1519597.7,1519595.00,1519595.98,1519596.37,1519597.00,1519598.00,1519599.00,1519599.82,1519600.31,1.302164e-06,-0.078652,0.000000,92.644159,1.821403e-06,0.059343,-1.201477,92.646534,92.641924,92.644130,92.642336,92.642770,92.643215,92.643670,92.644594,92.645066,92.645547,92.646035,0.000015,-0.000052,6.250000e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
1,3.045252e+07,30.762469,-0.101150,-1.311308,30452527.0,30452507.0,30452518.0,30452509.9,30452512.0,30452513.0,30452514.0,30452521.0,30452522.0,30452523.0,30452524.0,1.821325e-07,-0.022472,0.125000,2.145324e+07,1.013079e+04,0.447268,-0.311287,21453511.0,21453025.0,21453230.5,21453126.0,21453146.0,21453174.1,21453201.6,21453261.0,21453302.6,21453342.6,21453366.7,0.000005,-2.269663,-0.045455,125.357354,6.379062e-09,0.245186,-0.644995,125.357550,125.357220,125.357350,125.357249,125.357270,125.357300,125.357336,125.357370,125.357390,125.357420,125.357470,6.371308e-07,-2.921348e-06,-3.977273e-07,1.519591e+06,3.380914,-0

In [11]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    nans = imputed_df.isna().sum(axis=0)
    print(nans[nans!=0])

Series([], dtype: int64)


In [12]:
imputed_df.shape

(51764, 108)